# Using Toolhouse for Tool Use on Together.AI infrastructure
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/orliesaurus/together-cookbook/blob/toolhouse-tool-use-cookbook/Tool_use_with_Toolhouse.ipynb)

This Colab notebook leverages Together's powerful AI models and Toolhouse's streamlined function calling toolkit to efficiently execute complex tasks and generate creative content.

[Toolhouse](https://app.toolhouse.ai/) is the first complete infrastructure platform for building, running and managing tool use.

With Toolhouse, you can equip your LLM with extra skills (also known as tools).

Some of the most popular tools include:
- scraping data from the web or social media
- generate images
- compile or execute code and return the values

This cookbook demonstrates how you can **equip LLMs running on** [Together.ai](https://together.ai/) - **with tools**, without the need for your to code or prompt these tools.

In this short demo, we'll show how we can equip an LLM to generate images from real-data.

We'll use Toolhouse with the model tagged `meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo` and hosted on Together's infrastructure.

This model is fine-tuned for effective and precise tool use.

# 1. Setup

Let's install the dependencies.
We will use the OpenAI SDK which is compatible with Together's API - as long as we set the base_url.

We're also installing Toolhouse's Python SDK to access pre-built, pre-hosted tools.

In [ ]:
!pip install toolhouse openai

In [ ]:
# Import packages
import os
from openai import OpenAI
from toolhouse import Toolhouse

To integrate Together and Toolhouse, you'll need to set up two environment variables: `TOGETHER_API_KEY` and `TOOLHOUSE_API_KEY`. Follow these steps to obtain your API keys:

* **Together API Key**: Get your key by visiting the [Together Console](https://api.together.ai/).
* **Toolhouse API Key**: Sign up for Toolhouse using [this link](https://join.toolhouse.ai) to receive $150 in credits. You will receive your API key as part of the onboarding step, and you can always, navigate to the [Toolhouse API Keys page](https://app.toolhouse.ai/settings/api-keys) to create and get an API key.
* **Install the X tool**: In your [Toolhouse dashboard](https://app.toolhouse.ai), click Install next to the "Search X" tool ([direct link](https://app.toolhouse.ai/store/search_x)).
* **Install the Image Generation tool**: In your [Toolhouse dashboard](https://app.toolhouse.ai), click Install next to the "Image Generation" tool ([direct link](https://app.toolhouse.ai/store/image_generation_flux)).



# Set up keys in Colab

Once you have signe-up and got both API keys, set them as environment variables to start using Together with Toolhouse.

To do that click on the Key icon in the left menu on Colab and click **"Add new secret".**

Create 2 keys and make sure their **name** is spelled like below:
```
TOGETHER_API_KEY
TOOLHOUSE_API_KEY
```

Enter the values for each key and switch on the **allow note book access"**

# 2. Setting up the SDKs

Since we've imported the SDKs already, let's initialize them and authenticate ourselves using the API keys we just set.

If you want to check again, your API keys should be now in the left sidebar of Google Colab after clicking the key icon.

In [ ]:
from google.colab import userdata
client = OpenAI(base_url = "https://api.together.xyz/v1", api_key=userdata.get("TOGETHER_API_KEY"))

th = Toolhouse(api_key=userdata.get('TOOLHOUSE_API_KEY'), provider="openai")

We will use a Meta model specificially the Instruct flavor:
- [meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
 model](https://docs.together.ai/docs/function-calling#supported-models).

This model is based on the Meta 3.1 70B model and was fine-tuned for tool use and function calling:

We can test that we can call the model hosted on Together by running the cell below. It should output some info about NYC.

In [ ]:
MODEL = "meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo"
response = client.chat.completions.create(
    model=MODEL,
    messages=[{"role": "user", "content": "tell me about new york"}],
)
print(response.choices[0].message.content)


Likewise, you can use the `th.get_tools()` function to display all of the Toolhouse tools you have installed:

In [ ]:
print('TOOLS AVAILABLE:')
for tool in th.get_tools():
    print(f"Name: {tool['function']['name']}")
    print(f"Type: {tool['type']}")
    print(f"Description: {tool['function']['description']}")

For this demo, we will be using two tools from Toolhouse's store.
- Search X
- Image generation

The first tool lets you search the X social network and the second tool lets you generate an image from a prompt and returns it to the LLM.

# 3. Configure Tool Calls

First we'll configure the user prompt to the LLM.
This is a basic python list with a dictionary within it that takes `role` and a `content`.

We set `role` to `user` because this is a command that we, the user, want the LLM to understand.

Doing so will allow us to give a specific command to the LLM. The LLM will look at what tools we have available and attempt to leverage them to achieve the task.

In [ ]:
# User message to the LLM
messages = [
  {
    "role": "user",
    "content": "Find the last three messages from the account named @togethercompute on X/Twitter and summarize them in one sentence. Make it funny!",
  }
]

We'll send this message to our LLM hosted on TogetherAI.

Because we have tools provided by Toolhouse we can skip having to code our own Search X parser.
The tools are hosted and maintained by the Toolhouse team and are optimized for LLM usage.

In [ ]:
# TogetherAI response with Toolhouse tools
response = client.chat.completions.create(
  model=MODEL,
  messages=messages,
  # Passes the tools to the model
  tools=th.get_tools(),
)

print("The LLM autonomously decides to use the following tool(s) to achieve its goal of finding data on X/Twitter")
print("Tool selected: ", response.choices[0].message.tool_calls[0].function.name)

As you can see from the output above, the LLM properly identified that we'd like to invoke the 'search_x' tool

You can see the arguments it generated by running the code below, it will break iut down by ID, type and function arguments:

In [ ]:
tools_called = response.choices[0].message.tool_calls
for tool_called in tools_called:
    print(f"ID: {tool_called.id}")
    print(f"Type: {tool_called.type}")
    print(f"Function: {tool_called.function}")
    print('\n')

# 4. Execute Tool Call

Now that we learned how the LLM can generate arguments to pass to a function we'll have to **run the function with those arguments**.

The tool gets executed via the `run_tools` command, with the parameters that were identified in the previous LLM call.


After that we will get the result, and append it to the context, the `messages` variable.

In [ ]:
tool_run = th.run_tools(response)
messages += tool_run

# Here's what the messages variable contains now
print(messages)

Let's see what our messages list looks like:

In [ ]:
for message in tool_run:
  print(f"Role: {message['role']}")
  if 'tool_calls' in message:
    print(f"Tool Calls: {message['tool_calls']}")
  if 'tool_call_id' in message:
    print(f"Tool Call ID: {message['tool_call_id']}")
    print(f"Content: {message['content']}")
    print('\n')

We're now going to pass the whole message chain to the LLM. The LLM will generate a proper response based on our initial prompt:

```
Find the last three messages from the account named @togethercompute on X/Twitter and summarize them in one sentence. Make it funny!
```

## 4.1 The Funny Summary

In [ ]:
summary_response = client.chat.completions.create(
  model=MODEL,
  messages=messages
)

print('LLM RESPONSE:', summary_response.choices[0].message.content)

Let's add this last response - a funny summary of three posts retrieved from X - to the history of messages.

To achieve this we're combining two arrays into a new variable called `new_messages`.

In [ ]:
new_messages = messages + [
  {
    "role": "assistant",
    "content": summary_response.choices[0].message.content,
  }
]

Next we're going to give a new command to the LLM - generate us an image from this funny caption using the tool provided by Toolhouse.

This tool uses Flux - an image model hosted on TogetherAI, then hosts it to a storage location on the img.toolhouse.ai subdomain.

> This shows you that a tool can be used to perform multiple action at once (image generation and hosting), but it's up to you - the user and creator of tools - how to set up every tool for best agentic use.

We've installed this tool in step 1.

In [ ]:

generate_image_request = new_messages + [
  {
    "role": "user",
    "content": "Generate an image from the funny summary"
  }
]

# Let's ask the LLM to generate the image
image_response = client.chat.completions.create(
  model=MODEL,
  messages=generate_image_request,
  tools=th.get_tools(),
)
tool_run = th.run_tools(image_response)

Now that the LLM generated an image and stored it, we just want it to let the user know that the image is ready and can be viewed.

We're going to - yet again - ask the LLM to take the output of the function call and generate a response.

In [ ]:
image_messages = generate_image_request + tool_run

last_response = client.chat.completions.create(
  model=MODEL,
  messages=image_messages,
  tools=th.get_tools(),
)

print('LLM RESPONSE:', last_response.choices[0].message.content)


And there you have it, if you have managed to run all the cells in the correct order you should see a message with a valid URL. If you click it you will be able to see the generated image! The image is pretty random and truly depends on what "funny summary" we generated in step 4.1